In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import base64
from matplotlib_venn import venn2, venn2_circles
import matplotlib as mpl
from ast import literal_eval
import plotly.express as px
import flask
import os
import dash
from dash import dash_table
from dash import dcc
import dash_bootstrap_components as dbc
from dash import html
from dash.dependencies import Input, Output

In [30]:
#token for access map
mapbox_access_token = 'pk.eyJ1IjoidGl5bzE4MyIsImEiOiJja3UzejE3aWEzNTR3MnFxbmNlOGszNzJuIn0.W3_siDx1yV1_n2OP7QR73g'

In [31]:
fast_food = pd.read_csv('fast_food.csv')
expend = pd.read_csv('expend.csv')
bar_plot = pd.read_csv('bar_plot.csv')
covid = pd.read_csv('covid.csv')
income = pd.read_csv('income2.csv')
topic_df = pd.read_csv('topic_df.csv')

In [32]:
topic_df

,review_id,categories01,categories02,categories03,city,Value
0,izyfRcTnjQo9T_BHu6o74w,cocktailbars,mexican,salad,Los Angeles,Service
1,wkZiqiEoWWExICr0W40ZKw,cocktailbars,mexican,salad,Los Angeles,Service
2,YsiWFH8ep-P68Z5fnFX1JQ,cocktailbars,mexican,salad,Los Angeles,Food
3,EkRomu_roAanTW5WClUq1A,gastropubs,burgers,newamerican,Los Angeles,Food
4,UpYX8N6AYM8ldOPBMSfNLg,gastropubs,burgers,newamerican,Los Angeles,Food
...,...,...,...,...,...,...
11252,i4MdP568H5DeF2Vu8oUPJA,pizza,chicken_wings,sandwiches,Downey,Service
11253,VqNkuDN3_6vikk2qI1OoHg,pizza,chicken_wings,sandwiches,Downey,Environment
11254,bEjx11JL_KwbX3XBNq2nhA,sandwiches,NaN,NaN,Bell Gardens,Service
11255,Mx16eUgtho7hxr27Qtc8Iw,sandwiches,NaN,NaN,Bell Gardens,Service


In [33]:
# list category that will use for analytics
kategori = ['sandwiches', 'pizza','tacos','burgers', 
            'hotdogs', 'donuts', 'nooodles', 
            'pancakes', 'chicken_wings','salad']

### Dictionary for Dropdown Menu

In [34]:
lists = []
for i in range(len(fast_food['city'].unique())):
    kota = fast_food[(fast_food['city'].isin([fast_food['city'].unique()[i]]))]
    values = pd.unique(kota[['categories01','categories02','categories03']].values.ravel('K'))
    lists.append(values)
    

In [35]:
lists[101]

array(['tradamerican', 'pizza', 'burgers', 'hotdogs', 'sandwiches',
       'juicebars', 'mexican', 'italian', nan, 'salad',
       'breakfast_brunch', 'coffee', 'delis'], dtype=object)

In [36]:
categories_val = [l.tolist() for l in lists]

In [37]:
# categories_val = set(categories_val)
category_resto = []
for i in range(len(fast_food['city'].unique())):
    cs = [x for x in categories_val[i] if x in kategori]
    category_resto.append(cs)

In [38]:
category_resto[101]

['pizza', 'burgers', 'hotdogs', 'sandwiches', 'salad']

In [39]:
for i in range(len(fast_food['city'].unique())):
    my_dict = dict(zip(fast_food['city'].unique(), category_resto))

In [40]:
dict(list(my_dict.items())[0:2])

{'Lancaster': ['sandwiches',
  'pizza',
  'tacos',
  'burgers',
  'hotdogs',
  'chicken_wings',
  'donuts',
  'salad'],
 'Quartz Hill': ['pizza', 'burgers', 'sandwiches', 'hotdogs']}

## Dash plot 1

In [41]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SUPERHERO],
                meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}])

all_options = my_dict

In [42]:
app.layout = dbc.Container([ 
    
    #line 1
    dbc.Row([
        #column 1
        dbc.Col([
            dbc.Row([
                html.Br(),
                
                dbc.Col(
                    dbc.Card([
                        dbc.CardBody(
                            html.H2('Analysis Fast Food Restaurant in Los Angeles County',
                                   className='text-center align-items-center text-white mb-4',
                                   style={'height': '65px', 'line-height': '65px'}) 
                        )
                    ]),
                width=12,
                align='center')
            ]),
            
            html.Br(),
            
            dbc.Row([
                dbc.Col([
                    dcc.Dropdown(id='dpdn-city', multi=False, value='Los Angeles',
                        options=[{'label':x, 'value':x}
                                  for x in sorted(all_options.keys())],
                        className='text-dark',
                        searchable=True)
                ], width={'size':12, 'offset':0, 'order':1},
            align='around')
            ]),
            
            html.Br(),
            
            dbc.Row([
                dbc.Col([
                    dcc.Dropdown(id='dpdn-category', multi=True, 
                                 value=['pizza'],
#                                 options=[{'label':x, 'value':x}
#                                   for x in sorted(kategori)],
                        className='text-dark',
                        searchable=False),
                ], width={'size':12, 'offset':0, 'order':2}, align='around')
            ])
        ], width={'size':8, 'offset':0}),
        
        #column 2
        dbc.Col([
            dcc.Dropdown(id="dpdn-expend",
             options=[
                 {"label": "percentage", "value": 'percentage'},
                 {"label": "calculation", "value": 'calculation'}],
             multi=False,
             value='percentage',
             className='text-white',
             style={'width': "100%", 
                    'color': 'black',
#                     'background-color': 'rgba(0,0,0,0)',
                    'padding-top': '5px',
                    'box-shadow': '0px 8px 16px 0px rgba(0,0,0,0.2)'}, # padding (top, right, bottom, left)
             ),
            
            html.Br(),
            
            dcc.Graph(style={'height': '180px'}, id='expend-figs', figure={})
            ], style={'border':'1px solid', 'border-radius': 10},
            width={'size':4, 'offset':0},
            align='center'),
    
    ]), 
        #xs=12, sm=12, md=12, lg=5, xl=5)
    
    html.Br(),
    
    #line 2
    dbc.Row([
        html.Br(),
        
        #column 1
        dbc.Col([
            
            html.Div(id='title-map', children=[]),
#             html.H6(id='subtitle-map', children=[]),
            
            dcc.Graph(style={'height': '600px'}, id='map-fig', 
                      config={'displayModeBar': False, 'scrollZoom': True}, figure={})
        ], width={'size':6, 'offset':0},
            align='center'),
        
        #column 2
        dbc.Col([
            dbc.Row([
                dbc.Col([
                    
                    html.Div(id='title-eco', children=[]),
                    
                    dbc.Row([
                        dbc.Col([
                            html.Div(id='eco-2015', children=[])
                        ], width={'size':4, 'offset':0},
                                align='between'),
                        dbc.Col([
                            html.Div(id='eco-2017', children=[])
                        ], style={'text-align': 'center'},
                                width={'size':4, 'offset':0},
                                align='between'),
                        dbc.Col([
                            html.Div(id='eco-2019', children=[])
                        ], style={'text-align': 'right'},
                                width={'size':4, 'offset':0},
                                align='between'),
                    ], className= "d-flex justify-content-between",
                        style={'border':'1px solid', 'border-radius': 10}),
                    
                    dbc.Row([
                        dbc.Col([
                            dcc.Graph(style={'height': '200px'}, id='eco-fig', figure={})
                            ], style={'border':'1px solid', 'border-radius': 10},
                            width={'size':12, 'offset':0},
                            align='center')
                        ])
                    ])     
            ]),   
        
                
            html.Br(),
            
            dbc.Row([
                dbc.Col([
                    
                    html.Div(id='title-venn', children=[]),
                    
                    html.Img(style={'height': '287.5px'}, className = 'align-self-center', id='venn-fig', src={})
#                     dcc.Graph(style={'height': '287.5px'}, id='venn-fig', figure={})
                    ], style={'padding-right': '100px'},
                    width={'size':6, 'order': 2, 'offset':0},
                    align='center'),
                
                dbc.Col([
                    
                    html.Div(id='title-price', children=[]),
                    
                    dcc.Graph(style={'height': '287.5px'}, id='price-fig',
                              config={'displayModeBar': False, 'scrollZoom': False}, figure={})
                    ], style={'border':'1px solid', 'border-radius': 10},
                    width={'size':6, 'order': 1, 'offset':0},
                    align='center')
                ])
        ])
    ]),
    
    html.Br(),
    
    #line 3
    dbc.Row([
        
        #column 1
        dbc.Col([
            
            html.Div(id='title-bar', children=[]),
            
            dcc.Graph(style={'height': '300px'}, id='bar-fig', 
                      config={'displayModeBar': False, 'scrollZoom': True}, figure={})
        ], 
            style={'border':'1px solid', 'border-radius': 5},
            width={'size':4, 'offset':0},
            align='center'),
        
        #column 2
        dbc.Col([
            
            html.Div(id='title-word', children=[]),
            
            dcc.Graph(style={'height': '300px'}, id='word-fig', figure={})
        ], 
            style={'border':'1px solid', 'border-radius': 5},
            width={'size':4, 'offset':0},
            align='center'),
        
        #column 3
        dbc.Col([
            
            html.Div(id='title-covid', children=[]),
            
            dcc.Graph(style={'height': '300px'}, id='covid-fig', figure={})
        ], 
            style={'border':'1px solid', 'border-radius': 5},
            width={'size':4, 'offset':0},
            align='center'),
        
     ], no_gutters=False)
    
], style={'padding': '30px 30px 30px 30px'}, fluid=True)


#-------------------------------------------------------------------------------------------------------------------

colors = {
    'transparent': 'rgba(0,0,0,0)',
    'greyis' : "rgba(236, 236, 235, 1)",
    'text_blue': '#7FDBFF'    
}


@app.callback(
    Output('dpdn-category', 'options'),
    Input('dpdn-city', 'value'))
def set_cities_options(selected_country):
    return [{'label': i, 'value': i} for i in all_options[selected_country]]


@app.callback(
    Output('dpdn-category', 'value'),
    Input('dpdn-category', 'options'))
def set_cities_value(available_options):
    options = available_options['value']
    return options

# Connect the Plotly graphs with Dash Components
@app.callback(
    Output(component_id='expend-figs', component_property='figure'),
    [Input(component_id='dpdn-expend', component_property='value')])
    
def expenditure(expends):
    print(expends)
    print(type(expends))

    expenditures = expend.copy()
    expenditures = expenditures[expenditures["type"] == expends]

    # Plotly Express
    fig = px.line(
        expenditures,
        x=expenditures['Year'],
        y=expenditures.columns[1:-1]
    )
    
    fig.update_layout( 
        margin=dict(
            l=1,
            r=1,
            b=1,
            t=3,
            pad=4
    ),
        xaxis_showgrid=False, 
        yaxis_showgrid=False,
        xaxis_title=None,
        yaxis_title=None,
        paper_bgcolor=colors["transparent"],
        plot_bgcolor=colors["transparent"],
        legend=dict(
            title='variable : ',
            orientation='v',
            yanchor="bottom",
            y=0,
            xanchor="right",
            x=1
        ),
        font=dict(
#             family="Courier New, monospace",
#             size=22,
            color='white')
    )
    
    fig.update_traces(line=dict(width=5))
#     fig.update_xaxes(showline=True, linewidth=1, linecolor='white', showticklabels=False)
#     fig.update_yaxes(showline=True, linewidth=1, linecolor='white', showticklabels=False)


#     fig.update_xaxes(visible=False, showticklabels=False)
#     fig.update_yaxes(visible=False, showticklabels=False)
    
    return fig



# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='title-map', component_property='children'),
    Output(component_id='map-fig', component_property='figure')],
    [Input(component_id='dpdn-city', component_property='value'),
    Input(component_id='dpdn-category', component_property='value')])

def update_map(cities, categories):
    
#     if len(categories) > 0 & len(cities) > 0:
    print(cities)
    print(type(cities))
    print(categories)
    print(type(categories))
    
    container = "Restaurant {} in {} City ".format(', '.join(categories).title(), cities)
        
    #dff = fast_food.copy()
    #careful with () or []
    dff = fast_food[(fast_food['city'].isin([cities]))] 
    dff = dff[(dff['categories01'].isin(categories)) |
            (dff['categories02'].isin(categories)) |
            (dff['categories03'].isin(categories))]
#     dff = dff[dff["city"] == cities]
#     dff = dff[[dff["categories01"] == categories] |
#             dff[dff["categories02"] == categories] |
#             dff[dff["categories03"] == categories]]
#     elif len(category) == 0 | len(city) == 0:
#         raise dash.exceptions.PreventUpdate

    px.set_mapbox_access_token(mapbox_access_token)
    fig = px.scatter_mapbox(dff, lat=dff["latitude"], lon=dff["longitude"], 
                            color=dff["score"], 
                            size=dff["rating"], 
                            color_continuous_scale=px.colors.sequential.Bluyl,
                            hover_name=dff['name'],
                            hover_data=dict(
                                category=True,
                                rating=True,
                                price=True,
                                city=False,
                                latitude=False, 
                                longitude=False),
                            size_max=20,zoom=12)
    
#     fig.update_layout(coloraxis_showscale=False)
    fig.update_layout( 
        margin=dict(
            l=1,
            r=3,
            b=1,
            t=3,
            pad=4
    ),
        paper_bgcolor=colors["transparent"],
        plot_bgcolor=colors["transparent"],
        autosize=True,
        legend=dict(
            title='variable : ',
            orientation='v',
            yanchor="top",
            y=1,
            xanchor="right",
            x=1
        ),
        font=dict(
#             family="Courier New, monospace",
#             size=22,
            color='white')
    )
    
    return container, fig

# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='title-eco', component_property='children'),
     Output(component_id='eco-2015', component_property='children'),
     Output(component_id='eco-2017', component_property='children'),
     Output(component_id='eco-2019', component_property='children'),
    Output(component_id='eco-fig', component_property='figure')],
    Input(component_id='dpdn-city', component_property='value')
)

def update_venn(cities):
    
    container = "{} Household Income".format(cities)
    
    incomes = income[(income['city_2'].isin([cities]))].reset_index(drop=True)
    
    eco15 = "2015 : ${}".format(incomes['2015'].sum())
    eco17 = "2017 : ${}".format(incomes['2017'].sum())
    eco19 = "2019 : ${}".format(incomes['2019'].sum())
    
    # Plotly Express
    fig = px.bar(
        incomes,
        x=incomes['city_1'].unique(),
        y=incomes.columns[3:6]
    )
    
    fig.update_layout( 
        margin=dict(
            l=2,
            r=2,
            b=3,
            t=3,
            pad=4
    ),
        xaxis_title=None,
        yaxis_title=None,
        xaxis_visible=False, 
        xaxis_showticklabels=False,
        paper_bgcolor=colors["transparent"],
        plot_bgcolor=colors["transparent"],
        legend=dict(
            title= None,
            orientation='h',
            yanchor="bottom",
            y=1,
            xanchor="left",
            x=0
        ),
        font=dict(
#             family="Courier New, monospace",
#             size=22,
            color='white')
    )
    
    return container, eco15, eco17, eco19, fig

    
    
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='title-venn', component_property='children'),
    Output(component_id='venn-fig', component_property='src')],
    Input(component_id='dpdn-city', component_property='value')
)

def update_venn(cities):
    container = "Delivery and Pickup Restaurant in {}".format(cities)
    
    # print the image_path to confirm the selection is as expected
    print('current image_path = {}'.format(cities))
    encoded_image = base64.b64encode(open('img/' + cities + '.png', 'rb').read())
    fig = 'data:image/png;base64,{}'.format(encoded_image.decode())
        
    return container, fig

# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='title-price', component_property='children'),
    Output(component_id='price-fig', component_property='figure')],
    [Input(component_id='dpdn-city', component_property='value'),
    Input(component_id='dpdn-category', component_property='value')])

def update_prices(cities, categories):
     
    container = "Food Price in {}".format(cities)
     
    prices = fast_food[(fast_food['city'].isin([cities]))] 
    prices = prices[(prices['categories01'].isin(categories)) |
            (prices['categories02'].isin(categories)) |
            (prices['categories03'].isin(categories))]
    
        # Plotly Express
    prices_bar = px.bar(
        prices,
        x=prices['price'].unique(),
        y=prices['price'].value_counts(),
    )
    
    prices_bar.update_layout( 
        margin=dict(
            l=2,
            r=2,
            b=3,
            t=5,
            pad=4
    ),
        xaxis_title=None,
        yaxis_title=None,
        paper_bgcolor=colors["transparent"], #"rgba(236, 236, 235, 1)"
        plot_bgcolor=colors["transparent"],
        legend=dict(
            title= None,
            orientation='h',
            yanchor="bottom",
            y=1,
            xanchor="left",
            x=0
        ),
        font=dict(
#             family="Courier New, monospace",
#             size=22,
            color='white')
    )
    
#     prices_bar.update_xaxes(visible=False, showticklabels=False)
#     prices_bar.update_yaxes(visible=False, showticklabels=False)
    
    return container, prices_bar
    
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='title-bar', component_property='children'),
    Output(component_id='bar-fig', component_property='figure')],
    Input(component_id='dpdn-city', component_property='value'))

def update_bar(cities):
    
    container = "Total Restaurant in {}".format(cities)
    
    bars_plot = bar_plot[(bar_plot['city'].isin([cities]))]
    
        # Plotly Express
    fig_bar = px.bar(
        bars_plot,
        x=bars_plot['categories'],
        y=bars_plot.columns[2:],
        hover_data=dict(
            categories=False)
    )
    
    fig_bar.update_layout( 
        margin=dict(
            l=2,
            r=2,
            b=3,
            t=15,
            pad=4
    ),
        xaxis_title=None,
#         yaxis_title=None,
        paper_bgcolor=colors["transparent"],
        plot_bgcolor=colors["transparent"],
        legend=dict(
            title= None,
            orientation='h',
            yanchor="bottom",
            y=1,
            xanchor="left",
            x=0
        ),
        font=dict(
#             family="Courier New, monospace",
#             size=22,
            color='white')
    )
    
#     fig_bar.update_xaxes(visible=False, showticklabels=False)
#     fig_bar.update_yaxes(visible=False, showticklabels=False)
    
    return container, fig_bar


@app.callback(
    [Output(component_id='title-word', component_property='children'),
    Output(component_id='word-fig', component_property='figure')],
    [Input(component_id='dpdn-city', component_property='value'),
    Input(component_id='dpdn-category', component_property='value')])

def update_prices(cities, categories):
     
    container = "Topic Review Restaurant {} in {}".format(', '.join(categories).title(), cities)
     
    review = topic_df[(topic_df['city'].isin([cities]))] 
    review = review[(review['categories01'].isin(categories)) |
            (review['categories02'].isin(categories)) |
            (review['categories03'].isin(categories))]
    
    vals = review['Value'].value_counts().sort_values(ascending=False).sort_values(ascending=False)
    review_fig = px.pie(review, 
                    values=vals, 
                    names=vals.index)
    
    review_fig.update_layout( 
        margin=dict(
            l=2,
            r=2,
            b=3,
            t=15,
            pad=4
    ),
        paper_bgcolor=colors["transparent"],
        plot_bgcolor=colors["transparent"],
        legend=dict(
            title= None,
            orientation='h',
            yanchor="bottom",
            y=1,
            xanchor="left",
            x=0
        ),
        font=dict(
#             family="Courier New, monospace",
#             size=22,
            color='white')
    )
    
    return container, review_fig

#Covid data plot
@app.callback(
    [Output(component_id='title-covid', component_property='children'),
    Output(component_id='covid-fig', component_property='figure')],
    Input(component_id='dpdn-city', component_property='value')
)

def covid_graph(city_covid):
    print(city_covid)
    print(type(city_covid))
    
    container = "Covid-19 in {}".format(city_covid)

    covids = covid.copy()
    covids = covids[covids["geo_merge"] == city_covid]
    covids = covids.sort_values('ep_date', ascending=True).reset_index(drop=True)
    
    y = covids.columns[3:4] | covids.columns[7:8]
    # Plotly Express
    fig = px.line(
        covids,
        x=covids['ep_date'],
        y=y,
        labels = False
    )
    
    fig.update_layout( 
        margin=dict(
            l=1,
            r=1,
            b=1,
            t=6,
            pad=4
    ),
        xaxis_title=None,
        yaxis_title=None,
        paper_bgcolor=colors["transparent"],
        plot_bgcolor=colors["transparent"],
        legend=dict(
            title='variable : ',
            orientation='v',
            yanchor="top",
            y=1,
            xanchor="right",
            x=1
        ),
        font=dict(
#             family="Courier New, monospace",
#             size=22,
            color='white')
    )
    
    fig.update_traces(line=dict(width=5))
    
#     fig.update_xaxes(showline=True, linewidth=1, linecolor='white', mirror=True)
#     fig.update_yaxes(showline=True, linewidth=1, linecolor='white', mirror=True)

#     fig.update_xaxes(visible=False, showticklabels=True)
#     fig.update_yaxes(visible=False, showticklabels=True)
    
    return container, fig


In [ ]:
if __name__=='__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Oct/2021 16:29:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 16:29:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 16:29:53] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 16:29:54] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 16:29:54] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 16:29:54] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 16:29:54] "GET /[object%20Object] HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 16:29:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 16:29:54] "POST /_dash-update-component HTTP/1.1" 200 -


percentage
<class 'str'>
current image_path = Los Angeles


127.0.0.1 - - [31/Oct/2021 16:29:55] "POST /_dash-update-component HTTP/1.1" 200 -
<ipython-input-42-3f6953159ef3>:599: FutureWarning:

Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead



Los Angeles
<class 'str'>
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Tio\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Tio\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Tio\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Tio\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Tio\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Tio\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Tio\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    re

127.0.0.1 - - [31/Oct/2021 16:29:55] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [31/Oct/2021 16:29:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 16:29:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 16:29:55] "POST /_dash-update-component HTTP/1.1" 200 -


Los Angeles
<class 'str'>
['pizza']
<class 'list'>


127.0.0.1 - - [31/Oct/2021 16:29:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 16:29:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 16:29:56] "POST /_dash-update-component HTTP/1.1" 200 -
